This notebook demonstrates some code I wrote to handle data from the Deniz lab nanodrop.

In [1]:
cd ~/dropbox/grad/deniz/charge_patterning/data_wrangling

C:\Users\Emily\dropbox\grad\deniz\charge_patterning\data_wrangling


In [2]:
import glob
import pandas as pd

import bokeh.plotting
import bokeh.io

bokeh.io.output_notebook()

import phase_diagram_analysis as pda
import bokeh_scatter

Loading BokehJS ...

In [3]:
file_list = glob.glob("data_copy/*.tsv")

This is what the data looks like straight off the instrument:

In [4]:
pd.read_csv(file_list[0], sep="\t").head(15)

,#,Sample ID,User name,Date and Time,1 (nm),1 (Abs),2 (nm),2 (Abs),3 (nm),3 (Abs),...,5 (nm),5 (Abs),6 (nm),6 (Abs),7 (nm),7 (Abs),8 (nm),8 (Abs),9 (nm),9 (Abs)
0,1,buffer,Deniz Lab,6/5/2019 12:34:11 PM,350,-0.002,600,0.001,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,2,buffer,Deniz Lab,6/5/2019 12:35:00 PM,350,0.002,600,0.000,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,3,buffer,Deniz Lab,6/5/2019 12:34:49 PM,350,0.002,600,0.001,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,4,buffer,Deniz Lab,6/5/2019 1:18:36 PM,350,0.002,600,0.002,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,5,buffer,Deniz Lab,6/5/2019 1:18:45 PM,350,-0.002,600,0.000,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,6,buffer,Deniz Lab,6/5/2019 1:18:53 PM,350,-0.002,600,-0.002,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
6,7,buffer,Deniz Lab,6/5/2019 2:00:04 PM,350,0.004,600,0.004,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
7,8,buffer,Deniz Lab,6/5/2019 1:59:46 PM,350,-0.001,600,0.001,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
8,9,buffer,Deniz Lab,6/5/2019 1:59:56 PM,350,0.002,600,0.003,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
9,10,buffer,Deniz Lab,6/5/2019 2:00:15 PM,350,0.003,600,0.003,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


Here's how I've tidied it:

In [5]:
# this takes under 7 seconds on average
df = pda.tidy_nanodrop_data(file_list)
df.head()

Dropped 101 buffer samples.


,Sample ID,Abs 350,Abs 600,Peptide,Peptide concentration (uM),RNA/Peptide Ratio,Date,Time
0,RG7_150_0,0.002,0.001,RG7,150.0,0.000,6/5/2019,2:02:29 PM
1,RG7_150_0.01,0.004,0.001,RG7,150.0,0.010,6/5/2019,2:04:08 PM
2,RG7_150_0,-0.001,0.002,RG7,150.0,0.000,6/5/2019,2:02:40 PM
3,RG7_150_0.01,0.007,0.003,RG7,150.0,0.010,6/5/2019,2:04:17 PM
4,RG7_150_0.025,0.019,0.005,RG7,150.0,0.025,6/5/2019,2:05:41 PM


In [6]:
# only plot data from the first day of collection
p = bokeh_scatter.scatter(
    data=df.loc[df["Date"]== "6/5/2019"],
    cat="Peptide",
    x="RNA/Peptide Ratio",
    x_axis_type="log",
    y="Abs 350",
    height=350,
    width=500,
)

p.legend.location = "top_left"
    
bokeh.io.show(p)